In [63]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import matplotlib.pyplot as plt 
%matplotlib inline

## Preparing Data

In [ ]:
with open('input.txt','r', encoding='utf-8') as f:
    text = f.read()

In [15]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [12]:
len(text)

1115394

In [17]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [26]:
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for i,s in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [29]:
decode(encode('i love deep learning'))

'i love deep learning'

In [31]:
data = torch.tensor(encode(text),dtype=torch.long)

In [34]:
len(data), data.dtype

(1115394, torch.int64)

In [37]:
n = int(len(data)*0.9)
train_data = data[:n]
val_data = data[n:]

In [38]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

In [140]:
xb,yb = get_batch('train')
print(xb.shape,yb.shape)
for i in range(batch_size-2):
    for j in range(block_size):
        print(xb[i,:j+1].tolist(),'-->',yb[i,j].item())

torch.Size([4, 8]) torch.Size([4, 8])
[56] --> 41
[56, 41] --> 46
[56, 41, 46] --> 39
[56, 41, 46, 39] --> 56
[56, 41, 46, 39, 56] --> 45
[56, 41, 46, 39, 56, 45] --> 43
[56, 41, 46, 39, 56, 45, 43] --> 42
[56, 41, 46, 39, 56, 45, 43, 42] --> 1
[51] --> 54
[51, 54] --> 43
[51, 54, 43] --> 58
[51, 54, 43, 58] --> 57
[51, 54, 43, 58, 57] --> 1
[51, 54, 43, 58, 57, 1] --> 58
[51, 54, 43, 58, 57, 1, 58] --> 53
[51, 54, 43, 58, 57, 1, 58, 53] --> 1


## Bigram Language Model

In [144]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
        
    def forward(self,idx,targets =None):
        logits = self.token_embedding_table(idx) # B,T,C
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits, loss
    
    def generate(self,idx,max_new_token):
        for _ in range(max_new_token):
            logits,_ = self(idx)
            logits =logits[:,-1,:] # B,C
            probs = F.softmax(logits,dim=-1)
            idx_next = torch.multinomial(probs,num_samples=1)
            idx = torch.cat((idx,idx_next),dim=1)
        return idx
            

In [155]:
m = BigramLanguageModel(vocab_size)

In [156]:
optimizor = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [157]:
batch_size = 32
for i in range(8000):
    xb,yb = get_batch('train')
    logits,loss = m(xb,yb)
    optimizor.zero_grad(set_to_none=True)
    loss.backward()
    optimizor.step()
    if i%1000==0:
        print(loss.item())

4.663458824157715
3.666710376739502
3.162177324295044
2.834440231323242
2.7264578342437744
2.584068536758423
2.597648859024048
2.396040201187134


In [158]:
idx = torch.zeros((1,1),dtype=torch.long)
print(decode(m.generate(idx,200)[0].tolist()))


I and wise jueatoure bl;IO imbthime ad ul:
Ay n pr buesy eiste
HNULICl thelstatro saw mis h id ho d,
Ant blore CENICLIDUSAnchindlly l fe dere sop,
Wilinejurinta f byo, athe
GI hishe Cnod wim! thin the
